In [76]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [77]:
!nvidia-smi

Sun Feb  9 08:05:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P0             26W /   70W |     120MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
%%writefile matrix_multiplication.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

#define CHECK_CUDA_CALL(err)                                                \
    {                                                                       \
        if (err != cudaSuccess)                                             \
        {                                                                   \
            fprintf(stderr, "CUDA error in file %s at line %d: %s\n",       \
                    __FILE__, __LINE__, cudaGetErrorString(err));           \
            exit(EXIT_FAILURE);                                             \
        }                                                                   \
    }

#define TILE_SIZE 32
#define COARSE_FACTOR 4
// Reduce number of blocks and lower number of blocks can run parallely - few threads handle more "units" of work

__global__
void matrix_multiplication_tiled_thread_coarsed(float *A, float* B, float *result, int rows_result, int col_result, int inner_dim){
    __shared__ float A_ds[TILE_SIZE][TILE_SIZE];
    __shared__ float B_ds[TILE_SIZE][TILE_SIZE];

    int bx = blockIdx.x; int tx = threadIdx.x;
    int by = blockIdx.y; int ty = threadIdx.y;

    int row = by * TILE_SIZE + ty;
    int col_offset = bx * TILE_SIZE * COARSE_FACTOR;

    float dot_product[COARSE_FACTOR];
    for (int i=0; i< COARSE_FACTOR; ++i){
        dot_product[i] = 0.0;
    }
    // Loop over all tiles
    for (int ph=0; ph < (inner_dim + TILE_SIZE - 1)/TILE_SIZE; ++ph ){

        //r = row, c = TILE_SIZE * ph + tx;
        if (row < rows_result && (TILE_SIZE*ph + tx) < inner_dim){
            A_ds[ty][tx] = A[row * inner_dim + TILE_SIZE * ph + tx];
        }
        else{
            A_ds[ty][tx] = 0.0;
        }

        for (int coarse_idx=0; coarse_idx<COARSE_FACTOR; ++coarse_idx){
            int col = col_offset + TILE_SIZE * coarse_idx + tx;
            //row = TILE_SIZE * ph + ty, c = col
            if ((TILE_SIZE * ph + ty) < inner_dim && col < col_result){
                B_ds[ty][tx] = B[(TILE_SIZE * ph + ty) * col_result + col];
            }
            else{
                B_ds[ty][tx] = 0.0;
            }

            __syncthreads();

            for (int k=0; k < TILE_SIZE; ++k){
                dot_product[coarse_idx] += A_ds[ty][k] * B_ds[k][tx];
            }
            __syncthreads();
        }
    }
    for (int coarse_idx=0; coarse_idx<COARSE_FACTOR; ++coarse_idx ) {
        int col = col_offset + TILE_SIZE * coarse_idx + tx;
        if (row < rows_result && col < col_result ){
            result[row * col_result + col] += dot_product[coarse_idx];
        }

    }
}


float* matrix_multiplication(float *h_a, float *h_b, int row_a, int col_a, int row_b, int col_b){
    float *d_a, *d_b, *d_result;
    int size_a = sizeof(float) * row_a * col_a;
    int size_b = sizeof(float) * row_b * col_b;
    int size_result = sizeof(float) * row_a * col_b;
    float *h_result = new float[size_result];

    //Allocate device memory
    cudaError_t err = cudaMalloc((void**) &d_a, size_a);
    CHECK_CUDA_CALL(err);
    err = cudaMalloc((void**) &d_b, size_b);
    CHECK_CUDA_CALL(err);
    err = cudaMalloc((void**) &d_result, size_result);
    CHECK_CUDA_CALL(err);

    //copy matrices to device
    err = cudaMemcpy(d_a, h_a, size_a, cudaMemcpyHostToDevice);
    CHECK_CUDA_CALL(err);
    err = cudaMemcpy(d_b, h_b, size_b, cudaMemcpyHostToDevice);
    CHECK_CUDA_CALL(err);


    int thread_x = TILE_SIZE;
    int thread_y = TILE_SIZE;
    dim3 block_dims(thread_x, thread_y, 1);
    int blocks_x = (col_b + thread_x * COARSE_FACTOR - 1)/(thread_x * COARSE_FACTOR);
    int blocks_y = (row_a + thread_y - 1)/thread_y;
    dim3 grid_dims(blocks_x, blocks_y, 1);


    matrix_multiplication_tiled_thread_coarsed<<<grid_dims, block_dims>>>(d_a, d_b, d_result, row_a, col_b, col_a);

    // copy result to host
    err = cudaMemcpy(h_result, d_result, size_result, cudaMemcpyDeviceToHost);
    CHECK_CUDA_CALL(err);

    //free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_result);

    return h_result;
}


void test_matrix_multiplication(){
    cout << "Running Test 1:\n";
    float *A = new float[6];
    float *B = new float[8];

    fill_n(A, 6, 1.0f);
    fill_n(B, 8, 1.0f);

    float *C = matrix_multiplication(A, B, 3, 2, 2, 4);

    for (int i=0; i< 3; ++i){
        for (int j=0; j<4; ++j){
            cout << C[i * 4 + j] << " ";
        }
        cout << "\n";
    }


    cout << "\nRunning test 2:\n";

    A = new float[50*50];
    B = new float[2500];

    for (int i=0; i < 50; ++i){
        A[i*50 + i] = 1;
        B[i*50 + i] = 1;
    }
    C = matrix_multiplication(A, B, 50, 50, 50, 50);
    for (int i=0; i< 50; ++i){
        for (int j=0; j<50; ++j){
            cout << C[i * 50 + j] << " ";
        }
        cout << "\n";
    }
}


int main(){
    test_matrix_multiplication();
}

Overwriting matrix_multiplication.cu


In [14]:
!sudo apt-get install python3-pybind11

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-pybind11 is already the newest version (2.9.1-2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [21]:
!nvcc -arch=sm_75 -o matmul matrix_multiplication.cu

In [22]:
!./matmul

Running Test 1:
2 2 2 2 
2 2 2 2 
2 2 2 2 

Running test 2:
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0

In [32]:
!pip install ninja


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 14.4 MB/s eta 0:00:00


In [68]:
%%writefile matrix_multiplication.cu
#include <iostream>
#include <cuda_runtime.h>
#include <torch/extension.h>
#include <torch/types.h>

using namespace std;

#define STRINGFY(str) #str
#define TORCH_BINDING_COMMON_EXTENSION(func) \
  m.def(STRINGFY(func), &func, STRINGFY(func));


#define CHECK_CUDA_CALL(err)                                                \
    {                                                                       \
        if (err != cudaSuccess)                                             \
        {                                                                   \
            fprintf(stderr, "CUDA error in file %s at line %d: %s\n",       \
                    __FILE__, __LINE__, cudaGetErrorString(err));           \
            exit(EXIT_FAILURE);                                             \
        }                                                                   \
    }

#define TILE_SIZE 32
#define COARSE_FACTOR 4
// Reduce number of blocks and lower number of blocks can run parallely - few threads handle more "units" of work

__global__
void matrix_multiplication_tiled_thread_coarsed(float *A, float* B, float *result, int rows_result, int col_result, int inner_dim){
    __shared__ float A_ds[TILE_SIZE][TILE_SIZE];
    __shared__ float B_ds[TILE_SIZE][TILE_SIZE];

    int bx = blockIdx.x; int tx = threadIdx.x;
    int by = blockIdx.y; int ty = threadIdx.y;

    int row = by * TILE_SIZE + ty;
    int col_offset = bx * TILE_SIZE * COARSE_FACTOR;

    float dot_product[COARSE_FACTOR];
    for (int i=0; i< COARSE_FACTOR; ++i){
        dot_product[i] = 0.0;
    }
    // Loop over all tiles
    for (int ph=0; ph < (inner_dim + TILE_SIZE - 1)/TILE_SIZE; ++ph ){

        //r = row, c = TILE_SIZE * ph + tx;
        if (row < rows_result && (TILE_SIZE*ph + tx) < inner_dim){
            A_ds[ty][tx] = A[row * inner_dim + TILE_SIZE * ph + tx];
        }
        else{
            A_ds[ty][tx] = 0.0;
        }

        for (int coarse_idx=0; coarse_idx<COARSE_FACTOR; ++coarse_idx){
            int col = col_offset + TILE_SIZE * coarse_idx + tx;
            //row = TILE_SIZE * ph + ty, c = col
            if ((TILE_SIZE * ph + ty) < inner_dim && col < col_result){
                B_ds[ty][tx] = B[(TILE_SIZE * ph + ty) * col_result + col];
            }
            else{
                B_ds[ty][tx] = 0.0;
            }

            __syncthreads();

            for (int k=0; k < TILE_SIZE; ++k){
                dot_product[coarse_idx] += A_ds[ty][k] * B_ds[k][tx];
            }
            __syncthreads();
        }
    }
    for (int coarse_idx=0; coarse_idx<COARSE_FACTOR; ++coarse_idx ) {
        int col = col_offset + TILE_SIZE * coarse_idx + tx;
        if (row < rows_result && col < col_result ){
            result[row * col_result + col] += dot_product[coarse_idx];
        }

    }
}


torch::Tensor matrix_multiplication(torch::Tensor A, torch::Tensor B) {
    // Check inputs are CUDA tensors
    TORCH_CHECK(A.device().is_cuda(), "A must be a CUDA tensor");
    TORCH_CHECK(B.device().is_cuda(), "B must be a CUDA tensor");
    TORCH_CHECK(A.dtype() == torch::kFloat32, "A must be float32");
    TORCH_CHECK(B.dtype() == torch::kFloat32, "B must be float32");

    int rows_a = A.size(0);
    int inner_dim = A.size(1);
    int cols_b = B.size(1);

    auto result = torch::zeros({rows_a, cols_b}, A.options());

    dim3 grid_dim(
        (cols_b + TILE_SIZE * COARSE_FACTOR - 1) / (TILE_SIZE * COARSE_FACTOR),
        (rows_a + TILE_SIZE - 1) / TILE_SIZE
    );
    dim3 block_dim(TILE_SIZE, TILE_SIZE);

    matrix_multiplication_tiled_thread_coarsed<<<grid_dim, block_dim>>>(
        A.data_ptr<float>(),
        B.data_ptr<float>(),
        result.data_ptr<float>(),
        rows_a,
        inner_dim,
        cols_b
    );

    // Check for kernel launch errors
    cudaError_t err = cudaGetLastError();
    CHECK_CUDA_CALL(err);

    return result;
}


PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  TORCH_BINDING_COMMON_EXTENSION(matrix_multiplication)
}

Overwriting matrix_multiplication.cu


In [69]:
import torch
import time
from torch.utils.cpp_extension import load
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

lib = load(
    name="matrix_multiplication",
    sources=["/content/matrix_multiplication.cu"]
)

2.5.1+cu124
True
12.4


/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [70]:
A = torch.ones(50, 50, dtype=torch.float32).cuda()
B = torch.ones(50, 50, dtype=torch.float32).cuda()
start_time = time.time()
C = lib.matrix_multiplication(A, B)
end_time = time.time()

In [71]:
print (C)

tensor([[50., 50., 50.,  ..., 50., 50., 50.],
        [50., 50., 50.,  ..., 50., 50., 50.],
        [50., 50., 50.,  ..., 50., 50., 50.],
        ...,
        [50., 50., 50.,  ..., 50., 50., 50.],
        [50., 50., 50.,  ..., 50., 50., 50.],
        [50., 50., 50.,  ..., 50., 50., 50.]], device='cuda:0')


In [74]:
A = torch.eye(50, 50, dtype=torch.float32).cuda()
B = torch.eye(50, 50, dtype=torch.float32).cuda()
start_time = time.time()
C = lib.matrix_multiplication(A, B)
end_time = time.time()

In [75]:
C

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')